## Import Libraries

In [44]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [45]:
import matplotlib.pyplot as plt
plt.ion()

In [46]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [47]:
from os import getcwd, path, pardir

## Compile and program project

In [48]:
project_path_s = [path.curdir, "TrustZone_Flash-S"]
project_path_s

['.', 'TrustZone_Flash-S']

In [49]:
project_path_ns = [path.curdir, "TrustZone_Flash-NS"]
project_path_ns

['.', 'TrustZone_Flash-NS']

In [91]:
atprogram(path.abspath(path.join(*project_path_ns)), verbose=2)
atprogram(path.abspath(path.join(*project_path_s)), verbose=2, erase=False)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:21021
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00008000. Size = 0x00000700.
[DEBUG] Memory segment base written at 0x00008700. Size = 0x00000064.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "program"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:21025
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Memory segment base written a

0

## Data Logging

In [69]:
live_plot = False

Create a figure for the plot.

In [70]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [71]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_trustzone_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment TrustZone Flash",
}

Stop criteria to pass to the logger:

In [72]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [92]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger.log(1000,stop_fn)
    data = dgilib.data

In [74]:
print(data)

Interfaces:
	 256:   power,   samples: 6255000
	  48:   gpio,    samples:   48005



# Store Data

In [75]:
import pickle
pickle.dump(data, open("trustzone_flash_logger_data.p", "wb"))

# Load Data

In [76]:
data = pickle.load(open("trustzone_flash_logger_data.p", "rb"))

## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [77]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [93]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

c:\users\erikw_000\documents\github\pydgilib\pydgilib_extra\dgilib_calculations.py:148: UserWarning: Corrected a start_index of 0 in power_and_time_per_pulse.
  "power_and_time_per_pulse.")


In [94]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [95]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(nsc_store_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(nsc_store_charge) - 1}")

MAX_NUM_BYTES: 5999


In [96]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [97]:
results = []
for y in [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    fig, grid = result.plot()
#     xlabel=f"{labels[result_type]['x']} " +
#            f"[{labels[result_type]['x_unit']}]",
#     ylabel=f"{labels[result_type]['y']} " +
#            f"[{labels[result_type]['y_unit']}]")
#     fig.canvas.set_window_title(
#         f"Residuals of {parameter_name}")
    fig.tight_layout()
    fig.show()
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5999
    # variables        = 2
    chi-square         = 1.6108e-08
    reduced chi-square = 2.6860e-12
    Akaike info crit   = -159829.141
    Bayesian info crit = -159815.742
[[Variables]]
    slope:      2.7656e-08 +/- 1.2219e-11 (0.04%) (init = 0)
    intercept:  4.8194e-06 +/- 4.2325e-08 (0.88%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 5999
    # variables        = 2
    chi-square         = 8.9632e-13
    reduced chi-square = 1.4946e-16
    Akaike info crit   = -218598.542
    Bayesian info crit = -218585.143
[[Variables]]
    slope:      1.1512e-09 +/- 9.1146e-14 (0.01%) (init = 0)
    intercept:  8.9236e-09 +/- 3.1572e-10 (3.54%) (init = 1)
[[Correlations]] (unrep

In [98]:
fig2 = plt.figure(figsize=(8, 6))
fig2.canvas.set_window_title("Analysis TrustZone Flash")

In [99]:
charge_color = 'r'
time_color = 'b'

In [100]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [101]:
lines = []
lines += ax1.plot(num_bytes, nsc_store_charge, charge_color+'-', label='TrustZone Flash Store Charge')
lines += ax1.plot(num_bytes, nsc_load_charge, charge_color+'--', label='TrustZone Flash Load Charge')
lines += ax2.plot(num_bytes, nsc_store_time, time_color+'-', label='TrustZone Flash Store Time')
lines += ax2.plot(num_bytes, nsc_load_time, time_color+'--', label='TrustZone Flash Load Time')
ax1.legend(handles=lines)
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
ax1.set_title(f"TrustZone Flash Store Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Flash Load Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Flash Store Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"TrustZone Flash Load Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [87]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(nsc_store_charge, open("trustzone_flash_store_charge.p", "wb"))
pickle.dump(nsc_load_charge, open("trustzone_flash_load_charge.p", "wb"))
pickle.dump(nsc_store_time, open("trustzone_flash_store_time.p", "wb"))
pickle.dump(nsc_load_time, open("trustzone_flash_load_time.p", "wb"))

## Write config file

In [14]:
import json

config = {}
config["name"] = "TrustZone Flash"
config["project_paths"] = [project_path_ns, project_path_s]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["TrustZone Flash Write"], 3: ["TrustZone Flash Read"]}, 
                      "result_types": ["Charge", "Time"],
                      "section_types": {"init": [], 
                                        "store": ["TrustZone Flash Read"],
                                        "load": ["TrustZone Flash Write"],
                                        "exit": []},
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "byte", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "byte", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": 1}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)